In [1]:
import os
import cv2
from glob import glob
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Flatten
from tensorflow.keras.models import Model

In [91]:
dataset = r"E:\python\segmentation\Computer Vision\Computer Vision\Data\braintomurarchive"

In [131]:
""" Global Parameters """
H = 256
W = 256

In [7]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [9]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [11]:
def decoder_block(input, skip_connection, num_filters):
    
    #print("The input shape before Conv2DTranspose is :", input.shape)
    
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(input)
    #print("The input shape after Conv2DTranspose is :", x.shape)
         
    x = Concatenate()([x, skip_connection])
    #print("The input shape after Concatination is :", x.shape)

    x = conv_block(x, num_filters)
    
    return x

In [13]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    #print(s1.shape, s2.shape, s3.shape, s4.shape) 
    #print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    #print(d4.shape)

    output = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    #print(output.shape)

    model = Model(inputs, output, name="UNET")

    return model

In [15]:
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_unet(input_shape)
    model.summary()



Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 64)         0         ['batch_normalization[0][

In [19]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [21]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [230]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [232]:
def load_dataset(dataset, split=0.2):
    images = sorted(glob(os.path.join(dataset, "images", "*.png")))
    masks = sorted(glob(os.path.join(dataset, "masks", "*.png")))

    split_size = int(len(images) * split)
    X_train, X_validation = train_test_split(images, test_size=split_size, random_state=42)
    y_train_mask, y_validation_mask = train_test_split(masks, test_size=split_size, random_state=42)

    X_train, X_test = train_test_split(X_train, test_size=split_size, random_state=42)
    y_train_mask, y_mask_test = train_test_split(y_train_mask, test_size=split_size, random_state=42)
    
    return (X_train, y_train_mask), (X_validation, y_validation_mask), (X_test, y_mask_test)

In [103]:
path = r"E:\python\segmentation\Computer Vision\Computer Vision\Segmentation project"

In [197]:
def read_images(path):
    #path = tf.io.read_file(path)
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x , (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

In [199]:
def read_mask(path):
    #path = tf.io.read_file(path)
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x , (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [201]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_images(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [203]:
def tf_dataset(X, y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [213]:
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    creat_dir(path + "/files")
    batch_size = 2
    learning_rate = 1e-4
    num_epochs = 10
    model_path = os.path.join(path + "files" + "brain_tumor_model.h5")
    csv_path = os.path.join(path + "files" + "csv_log.csv")
    (X_train, y_train_mask), (X_validation, y_validation_mask), (X_test, y_mask_test) = load_dataset(dataset)

    print(f"Train : {len(X_train)} - {len(y_train_mask)}")
    print(f"Validation : {len(X_validation)} - {len(y_validation_mask)}")
    print(f"Test : {len(X_test)} - {len(y_mask_test)}")

    train_dataset = tf_dataset(X_train, y_train_mask, batch=batch_size)
    validation_dataset = tf_dataset(X_validation, y_validation_mask, batch=batch_size)

    #for x, y in train_dataset:
        #print (x.shape, y.shape)
    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(learning_rate=learning_rate), metrics=[dice_coef])
    callbacks = [

    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=validation_dataset,
        callbacks=callbacks
    )

Train : 1840 - 1840
Validation : 612 - 612
Test : 612 - 612
Epoch 1/10
920/920 [==============================] - ETA: 0s - loss: 0.7326 - dice_coef: 0.2674
Epoch 1: val_loss improved from inf to 0.58872, saving model to E:\python\segmentation\Computer Vision\Computer Vision\Segmentation projectfilesbrain_tumor_model.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


920/920 [==============================] - 1993s 2s/step - loss: 0.7326 - dice_coef: 0.2674 - val_loss: 0.5887 - val_dice_coef: 0.4113 - lr: 1.0000e-04
Epoch 2/10
920/920 [==============================] - ETA: 0s - loss: 0.5004 - dice_coef: 0.4996
Epoch 2: val_loss improved from 0.58872 to 0.50470, saving model to E:\python\segmentation\Computer Vision\Computer Vision\Segmentation projectfilesbrain_tumor_model.h5
920/920 [==============================] - 1935s 2s/step - loss: 0.5004 - dice_coef: 0.4996 - val_loss: 0.5047 - val_dice_coef: 0.4953 - lr: 1.0000e-04
Epoch 3/10
920/920 [==============================] - ETA: 0s - loss: 0.4201 - dice_coef: 0.5799
Epoch 3: val_loss improved from 0.50470 to 0.42285, saving model to E:\python\segmentation\Computer Vision\Computer Vision\Segmentation projectfilesbrain_tumor_model.h5
920/920 [==============================] - 1937s 2s/step - loss: 0.4201 - dice_coef: 0.5799 - val_loss: 0.4228 - val_dice_coef: 0.5772 - lr: 1.0000e-04
Epoch 4/10
9

In [220]:
model.save("brain_tumor.h5")

In [222]:
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score

In [242]:
def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255

    #print(image.shape, mask.shape, y_pred.shape)
    cat_images = np.concatenate([image, line, mask, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [254]:
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(path + "result")
    my_model = "brain_tumor.h5"
    with CustomObjectScope({"dice_coef":dice_coef, "dice_loss":dice_loss}):
        loaded_model = tf.keras.models.load_model(my_model)

    loaded_model.summary()

    (X_train, y_train_mask), (X_validation, y_validation_mask), (X_test, y_mask_test) = load_dataset(dataset)

    score = []
    for x, y in tqdm(zip(X_test, y_mask_test), total=len(y_mask_test)):
        name = x.split("/")[-1]
        print(name)

        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (W, H))
        x = image / 255.0
        #x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
    
        mask = cv2.imread(y, cv2.IMREAD_COLOR)
        mask = cv2.resize(mask, (W, H))
                                
        y_pred = loaded_model.predict(x, verbose=0)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        #print(y_pred.shape)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)

        save_image_path = os.path.join("result", name)
        #save_results(image, mask, y_pred, save_image_path)

        mask = mask / 255.0
        mask = (mask > 0.5).astype(np.int32).flatten()
        y_pred = y_pred.flatten()

        f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary")

        score.append([name, f1_value, jac_value, recall_value, precision_value])
    
    score = [s[1:] for s in score]
    score = np.mean(score, axis=0)
    print(f"F1: {score[0]:0.5f}")
    print(f"Jaccard: {score[1]:0.5f}")
    print(f"Recall: {score[2]:0.5f}")
    print(f"Precision: {score[3]:0.5f}")

    df = pf.DataFrame(score, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")
    

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_152 (Conv2D)         (None, 256, 256, 64)         1792      ['input_9[0][0]']             
                                                                                                  
 batch_normalization_144 (B  (None, 256, 256, 64)         256       ['conv2d_152[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_144 (Activation  (None, 256, 256, 64)         0         ['batch_normalization_144[0

  0%|                                                                                          | 0/612 [00:00<?, ?it/s]

E:\python\segmentation\Computer Vision\Computer Vision\Data\braintomurarchive\images\833.png


  0%|▏                                                                                 | 1/612 [00:03<38:51,  3.82s/it]

E:\python\segmentation\Computer Vision\Computer Vision\Data\braintomurarchive\images\2069.png


...

100%|███████████████████████████████████████████████████████████████████████████████▋| 610/612 [06:28<00:01,  1.59it/s]

E:\python\segmentation\Computer Vision\Computer Vision\Data\braintomurarchive\images\2667.png


100%|███████████████████████████████████████████████████████████████████████████████▊| 611/612 [06:28<00:00,  1.59it/s]

E:\python\segmentation\Computer Vision\Computer Vision\Data\braintomurarchive\images\367.png


100%|████████████████████████████████████████████████████████████████████████████████| 612/612 [06:29<00:00,  1.57it/s]
